In [1]:
! gpustat

gpu20                    Sun Apr 21 18:21:31 2024  550.54.15
[0] Tesla V100-DGXS-32GB | 50°C,   6 % |  7081 / 32768 MB | tanshiyin(4614M) kwonjingun(2464M)
[1] Tesla V100-DGXS-32GB | 61°C, 100 % | 10883 / 32768 MB | zym(10880M)
[2] Tesla V100-DGXS-32GB | 48°C,   0 % |  1045 / 32768 MB | faza.thirafi(1042M)
[3] Tesla V100-DGXS-32GB | 45°C,   0 % |     3 / 32768 MB |


In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
os.environ['TORCH_USE_CUDA_DSA'] = '1'
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

cores = os.cpu_count()
cores


import torch

# 检查 GPU 是否可用
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"发现 {device_count} 个可用的 GPU:")
    for i in range(device_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    device = torch.device('cuda')
else:
    print("未发现可用的 GPU")
    device = torch.device('cpu')

发现 1 个可用的 GPU:
GPU 0: Tesla V100-DGXS-32GB


In [5]:
from primer_hf_main_modify import PRIMERSummarizer
from transformers import (
    AutoTokenizer,
    LEDForConditionalGeneration,
)
import argparse

In [8]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __setattr__(self, key, value):
        self.__dict__[key] = value

    def __getattr__(self, item):
        return f"Attribute '{item}' not found."
    
    def __getitem__(self, key):
        try:
            return self.__dict__[key]
        except KeyError:
            return f"Attribute '{key}' not found."
    
    def __setitem__(self, key, value):
        self.__dict__[key] = value
    
    def __str__(self): # 作用于print输出
        attrs = [f"{key}={value}" for key, value in self.__dict__.items()]
        return f"Namespace({', '.join(attrs)})"
    
    def __repr__(self): # 作用于jupyter notebook表示
        attrs = [f"{key}={value}" for key, value in self.__dict__.items()]
        return f"Namespace({', '.join(attrs)})"

In [9]:
def str_to_bool(value):
    if isinstance(value, bool):
        return value
    if value.lower() in {'false', 'f', '0', 'no', 'n'}:
        return False
    elif value.lower() in {'true', 't', '1', 'yes', 'y'}:
        return True
    raise ValueError(f'{value} is not a valid boolean value')

parser = argparse.ArgumentParser()

########################
# Gneral
parser.add_argument("--devices", default=0, type=int, help="number of gpus to use")
parser.add_argument(
    "--accelerator", default='gpu', type=str, help="Type of accelerator"
) # gpu
parser.add_argument("--mode", default="train", choices=["train", "test"])
parser.add_argument(
    "--model_name", default="primer",
)
parser.add_argument(
    "--primer_path", type=str, default="allenai/PRIMERA-multinews", # ../PRIMERA/ # allenai/PRIMERA
)
parser.add_argument("--join_method", type=str, default="tsy_design") # concat_start_wdoc_global
parser.add_argument(
    "--debug_mode", action="store_true", help="set true if to debug"
)
parser.add_argument(
    "--compute_rouge",
    action="store_true",
    help="whether to compute rouge in validation steps",
)
parser.add_argument(
    "--saveRouge",
    action="store_true",
    help="whether to compute rouge in validation steps",
)

parser.add_argument("--progress_bar_refresh_rate", default=1, type=int)
parser.add_argument("--model_path", type=str, default="./run_saves/tsy_join_method/") # "./pegasus/"
parser.add_argument("--ckpt_path", type=str, default=None)
parser.add_argument("--saveTopK", default=3, type=int)
parser.add_argument(
    "--resume_ckpt",
    type=str,
    help="Path of a checkpoint to resume from",
    default=None,
)

parser.add_argument("--data_path", type=str, default="../dataset/")
parser.add_argument("--dataset_name", type=str, default="multi_news") # arxiv
parser.add_argument(
    "--num_workers",
    type=int,
    default=4,
    help="Number of workers to use for dataloader",
)

parser.add_argument("--batch_size", default=4, type=int)
parser.add_argument("--max_length_input", default=4096, type=int)
parser.add_argument("--max_length_tgt", default=1024, type=int)
parser.add_argument("--min_length_tgt", default=0, type=int)
parser.add_argument("--label_smoothing", type=float, default=0.0, required=False)
parser.add_argument(
    "--adafactor", action="store_true", help="Use adafactor optimizer"
)
parser.add_argument(
    "--grad_ckpt",
    action="store_true",
    help="Enable gradient checkpointing to save memory",
)
parser.add_argument(
    "--rand_seed",
    type=int,
    default=0,
    help="seed for random sampling, useful for few shot learning",
)

########################
# TSY added
parser.add_argument("--permute_docs", type=str_to_bool, default=False)

########################
# For training
parser.add_argument(
    "--pretrained_model_path", type=str, default="./pretrained_models/",
)
parser.add_argument(
    "--limit_valid_batches", type=int, default=None,
)
parser.add_argument("--lr", type=float, default=3e-5, help="Maximum learning rate")
parser.add_argument(
    "--warmup_steps", type=int, default=1000, help="Number of warmup steps"
)
parser.add_argument(
    "--accum_data_per_step", type=int, default=16, help="Number of data per step"
)
parser.add_argument(
    "--total_steps", type=int, default=-1, help="Number of steps to train" # 50000
)
parser.add_argument(
    "--num_train_data",
    type=int,
    default=-1,
    help="Number of training data, -1 for full dataset and any positive number indicates how many data to use",
)

parser.add_argument(
    "--fix_lr", action="store_true", help="use fix learning rate",
)
parser.add_argument(
    "--test_imediate", action="store_true", help="test on the best checkpoint",
)
parser.add_argument(
    "--fewshot",
    action="store_true",
    help="whether this is a run for few shot learning",
)
########################
# For testing
parser.add_argument(
    "--limit_test_batches",
    type=int,
    default=None,
    help="Number of batches to test in the test mode.",
)
parser.add_argument("--beam_size", type=int, default=1, help="size of beam search")
parser.add_argument(
    "--length_penalty",
    type=float,
    default=1,
    help="length penalty of generated text",
)
parser.add_argument(
    "--mask_num",
    type=int,
    default=0,
    help="Number of masks in the input of summarization data",
)
parser.add_argument(
    "--test_batch_size",
    type=int,
    default=-1,
    help="batch size for test, used in few shot evaluation.",
)
parser.add_argument(
    "--applyTriblck",
    action="store_true",
    help="whether apply trigram block in the evaluation phase",
)

# args = parser.parse_args()  # Get pad token id
args, unknown_args = parser.parse_known_args()

## Convert to HF model

In [22]:
resume_ckpt = "run_saves/tsy_join_method_train_5/summ_checkpoints/step=33732-vloss=2.03-avgr=0.3146-v1.ckpt"
output_dir = "./run_saves/hf_form_for_evaluate/"

In [12]:
model = PRIMERSummarizer.load_from_checkpoint(resume_ckpt, args=args)

/home/lr/tanshiyin/anaconda3/envs/mds/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [15]:
model.model
model.tokenizer

LEDTokenizerFast(name_or_path='allenai/PRIMERA-multinews', vocab_size=50265, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<doc-sep>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
	50265: AddedToken("<doc-sep>", rstrip=False, lstri

In [23]:
model.model.save_pretrained(output_dir)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'num_beams': 5, 'no_repeat_ngram_size': 3}


In [25]:
model.tokenizer.save_pretrained(output_dir)

('./run_saves/hf_form_for_evaluate/tokenizer_config.json',
 './run_saves/hf_form_for_evaluate/special_tokens_map.json',
 './run_saves/hf_form_for_evaluate/vocab.json',
 './run_saves/hf_form_for_evaluate/merges.txt',
 './run_saves/hf_form_for_evaluate/added_tokens.json',
 './run_saves/hf_form_for_evaluate/tokenizer.json')

## Evaluate the Model

In [5]:
from transformers import (
    AutoTokenizer,
    LEDForConditionalGeneration,
    AutoModelForSeq2SeqLM,
)
from datasets import load_dataset, load_metric
from tqdm.notebook import tqdm

rouge = load_metric("rouge")

/home/lr/tanshiyin/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1909548/2874987259.py:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/home/lr/tanshiyin/anaconda3/envs/mds/lib/python3.9/site-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to l

In [11]:

PRIMER_path = "./run_saves/hf_form_for_evaluate/"

TOKENIZER = AutoTokenizer.from_pretrained(PRIMER_path)
# MODEL = LEDForConditionalGeneration.from_pretrained(PRIMER_path)
MODEL = LEDForConditionalGeneration.from_pretrained(PRIMER_path).to(device)

MODEL.gradient_checkpointing_enable()
PAD_TOKEN_ID = TOKENIZER.pad_token_id
DOCSEP_TOKEN_ID = TOKENIZER.convert_tokens_to_ids("<doc-sep>")
print(PAD_TOKEN_ID, DOCSEP_TOKEN_ID)

1 50265


In [12]:
dataset=load_dataset('multi_news')

In [16]:
import time

def process_document(documents):
    input_ids_all=[]
    for data in documents:
        all_docs = data.split("|||||")
        for i, doc in enumerate(all_docs):
            doc = doc.replace("\n", " ")
            doc = " ".join(doc.split())
            all_docs[i] = doc

        #### concat with global attention on doc-sep
        input_ids = []
        for doc in all_docs:
            input_ids.extend(
                TOKENIZER.encode(
                    doc,
                    truncation=True,
                    max_length=4096 // len(all_docs),
                )[1:-1]
            )
            input_ids.append(DOCSEP_TOKEN_ID)
        input_ids = (
            [TOKENIZER.bos_token_id]
            + input_ids
            + [TOKENIZER.eos_token_id]
        )
        input_ids_all.append(torch.tensor(input_ids))
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids_all, batch_first=True, padding_value=PAD_TOKEN_ID
    )
    return input_ids


def batch_process(batch):
    input_ids=process_document(batch['document'])
    
    
    input_ids = input_ids.to(device)
    # get the input ids and attention masks together
    global_attention_mask = torch.zeros_like(input_ids).to(input_ids.device)
    # put global attention on <s> token

    global_attention_mask[:, 0] = 1
    global_attention_mask[input_ids == DOCSEP_TOKEN_ID] = 1

    start_time = time.time()
    generated_ids = MODEL.generate(
        input_ids=input_ids,
        global_attention_mask=global_attention_mask,
        use_cache=False,
        max_length=1024,
        num_beams=5,
    )
    end_time = time.time()
    print("generate time: ", end_time - start_time)
    
    generated_str = TOKENIZER.batch_decode(
            generated_ids.tolist(), skip_special_tokens=True
        )

    result={}
    result['generated_summaries'] = generated_str
    result['gt_summaries']=batch['summary']
    return result

In [17]:
import random
data_idx = random.choices(range(len(dataset['test'])),k=10)
# data_idx = range(len(dataset['test']))
dataset_small = dataset['test'].select(data_idx)
result_small = dataset_small.map(batch_process, batched=True, batch_size=2)

Map:  20%|██        | 2/10 [00:58<03:55, 29.43s/ examples]Input ids are automatically padded from 614 to 1024 to be a multiple of `config.attention_window`: 512


generate time:  58.794923305511475


Map:  40%|████      | 4/10 [01:10<01:33, 15.57s/ examples]Input ids are automatically padded from 3408 to 3584 to be a multiple of `config.attention_window`: 512


generate time:  11.716407060623169


Map:  60%|██████    | 6/10 [04:23<03:29, 52.40s/ examples]Input ids are automatically padded from 1331 to 1536 to be a multiple of `config.attention_window`: 512


generate time:  192.3845944404602


Map:  80%|████████  | 8/10 [05:08<01:21, 40.61s/ examples]Input ids are automatically padded from 1612 to 2048 to be a multiple of `config.attention_window`: 512


generate time:  44.86605954170227


Map: 100%|██████████| 10/10 [06:19<00:00, 37.91s/ examples]

generate time:  70.8754575252533


In [18]:
score=rouge.compute(predictions=result_small["generated_summaries"], references=result_small["gt_summaries"], use_stemmer=True)
print(score['rouge1'].mid)
print(score['rouge2'].mid)
print(score['rougeL'].mid)

Score(precision=0.44424478932154754, recall=0.5022607729006733, fmeasure=0.45485686507700906)
Score(precision=0.13797548168492504, recall=0.14655468322949716, fmeasure=0.1373016405017336)
Score(precision=0.19899726584511804, recall=0.22245682634806052, fmeasure=0.20221953006844948)


In [6]:
15210/317700

0.047875354107648725